In [1]:
import cPickle as pickle
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
import numpy as np
import sys
import os.path
sys.path.append(os.path.join('', 'TFIDF'))
sys.path.append(os.path.join('', 'util'))
from timer import Timer

In [ ]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [ ]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))
questions = questions[:40000]

words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.encode('ascii', 'ignore').lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1
    
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1

word_indices = list(word_indices.iteritems())

output = open('wordIndices_md.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

output = open('questionTexts_md.pkl', 'wb')
pickle.dump(question_texts, output, -1)
output.close()

In [9]:
try:
     tfidf = reload(tfidf)
except NameError:
    import TFIDF_numpy as tfidf

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))
    
tfidf.init_globals()
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.init_tfs()
serial_tfs = tfidf.create_tfs()
serial_idf = tfidf.calculate_idf()
serial_tfidfs = tfidf.calculate_tfidfs()
tfidf.calculate_tfidf_norms()
# tfidf.calculate_cossim(question_texts[0])

Time for initialization: 4.05311584473e-06
Time for load_questions: 9.53674316406e-07
Time for load_indices: 0.0175580978394
Time for init_tfs: 5.42286801338
Time for create_tfs: 4.17049908638
Time for create_idf: 2.29096007347
Time for calculate_tfidfs: 4.3447060585
Time for calculate_tfidf_norms: 9.97682905197


In [2]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(4)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = tfidf_c.create_tfs()
cython_idf = tfidf_c.calculate_idf()
tfidf_c.init_tfidfs()
cython_tfidfs = tfidf_c.calculate_tfidfs()
# tfidf.calculate_cossim(question_texts[0])

Compiling with /usr/local/bin/clang-omp
Time for Initialization: 1.4066696167e-05
Time for load_questions: 0.279912948608
Time for load_indices: 0.0315628051758
Time for init_tfs: 3.32865476608
Time for create_tfs: 0.292266845703
Time for calculate_idf: 0.285604000092
Time for int_tfidfs: 3.25207996368
Time for calculate_tfidfs: 0.477329969406


In [4]:
# Verification
print np.linalg.norm(np.asarray(cython_tfs) - serial_tfs)
print np.linalg.norm(np.asarray(cython_idf) - serial_idf)
print np.linalg.norm(np.asarray(cython_tfidfs) - serial_tfidfs)

0.0
5.55101584587e-05
0.00134948606118


In [ ]:
np.asarray(cython_tfs).index(983.57928)

In [ ]:
np.where(np.asarray(cython_tfs) == 983.57928)

In [ ]:
serial_tfs[6378][18644]

In [ ]:
for (key, value) in word_indices :
    if value == 18644 :
        print key

In [ ]:
np.sum(serial_idf - np.asarray(cython_idf))

In [ ]:
dir(tfidf_c)

In [ ]:
with Timer() as t :
    a = list(word_indices.iteritems())
print t.interval
with Timer() as t :
    b = dict(a)
print t.interval

In [3]:
import xxhash

In [11]:
word_index = dict(word_indices)
simhashes = []
hi = 0
num_words = 0
for u in range(len(question_texts)):
    W = np.zeros(64)
    for i in range(len(question_texts[u])):
#         print (question_texts[u][i])
        try :
            num_words += 1
            word = question_texts[u][i]
            wordhash = xxhash.xxh64(word).intdigest()
            counter = 63
            while wordhash > 0:
                bit = wordhash % 2
                if bit:
                    W[counter] += serial_tfidfs[u][word_index[word]]
                else:
                    W[counter] -= serial_tfidfs[u][word_index[word]]
                wordhash = wordhash >> 1
                counter -= 1
        except KeyError :
            hi += 1
    simhash = 0
    for i in range(len(W)):
        if W[i] >= 0:
            simhash += 1
        if i < len(W)-1:
            simhash = simhash << 1
    simhashes.append(simhash)
print hi
print num_words

38993
2042631


[18000146472455003855L, 16578382763199721004L, 16292957576427023093L, 18039012055662902833L, 14562213533534886821L, 14081310540723043621L, 14439937281953709056L, 16091881284724928646L, 16518443712630013376L, 16932501913554116675L, 18371260430517757739L, 13993980948572716914L, 10286212632020753019L, 16061475621723247953L, 14151088806118324679L, 14781206710663361623L, 16976056787935297538L, 16616941922442359193L, 13906874016067134219L, 18337404123313394810L, 17576002075635197437L, 18354286836012111839L, 17436806179704257963L, 17125761421099912392L, 14585443134982071089L, 17704555120269141832L, 14428899903456974178L, 11537176900935610093L, 16321290119948479821L, 17505652996823410476L, 18045587288868837006L, 13669475813502688533L, 17049668571634642081L, 17522500336430286889L, 17549410204944696698L, 14237608249183254926L, 14570057780026850722L, 17930294754385672527L, 12258572623181674142L, 13081081268678926744L, 14872035615288331325L, 18108497175568767353L, 12856761257958088825L, 1794724921

In [ ]:
for i in xrange(len(simhashes)):
    for j in xrange(i, len(simhashes)):
        if i != j:
            

In [ ]:
numpy_questions

In [ ]:
np.array(list(word_indices.iteritems()))

In [ ]:
a = np.array(list(word_indices.iteritems()))
a[:,0]

In [ ]:
word_indices = pickle.load(open('wordIndices.pkl', 'rb'))

In [ ]:
len(word_indices)

In [ ]:
"%".encode('utf-8')